In [3]:
import json
import os
import numpy as np
import re
import nltk
import stanza
import torch
from transformers import BartForCausalLM, BartTokenizer
import inspect

from utils import *

In [89]:
stanza_tokenizer = stanza.Pipeline("en")

2024-06-12 15:33:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-06-12 15:33:40 INFO: Downloaded file to /home/nchibbar/stanza_resources/resources.json
2024-06-12 15:33:41 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-06-12 15:33:41 INFO: Using device: cuda
2024-06-12 15:33:41 INFO: Loading: tokenize
2024-06-12 15:33:42 INFO: Loading: mwt
2024-06-12 15:33:42 INFO: Loading: pos
2024-06-12 15:33:43 INFO: Loading: lemma
2024-06-12 15:33:43 INFO: Loading: constituency
2024-06-12 15:33:43 INFO: Loading: depparse
2024-06-12 15:33:43 INFO: Loading: sentiment
2024-06-12 15:33:44 INFO: Loading: ne

In [19]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [2]:
sample_text = """Hey there! 😊 Have you checked out the new website? Visit us at https://www.example.com or email us at info@example.com for more details. #ExcitingNews 📰

The meeting is scheduled for 12:30 PM on 15th August, 2023. Don't be late! 🚀 We’ll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%. 📊

Here are the main points:

Increase marketing budget by 25% 🚀
Launch the new product line in Q4 🎉
Expand into the APAC region 🌏

To-do List:

 Finalize the budget proposal
 Review the Q3 report
 Prepare for the product launch event

Alice said, “We need to push our limits!” while Bob replied, “Let’s ensure quality over quantity.” This debate has been ongoing for weeks now. Some say, "Quality is key," but others argue, "We need quantity to drive growth."

Contact our support team at support@example.com or call 1-800-555-0199. For international calls, dial +1-800-555-0199.

Here's a snippet of code we need to review:

def hello_world():
    print("Hello, World!")

Make sure to check it and ensure it’s compliant with PEP 8 standards. Also, we have some legacy code:

<!DOCTYPE html>
<html>
<body>
    <h1>My First Heading</h1>
    <p>My first paragraph.</p>
</body>
</html>

Random thoughts:

I can't believe it's almost the end of the year!
What are everyone's New Year's resolutions? 🎆
The weather has been crazy lately, right? 🌦️
Check out this great article: https://www.example.com/blog/awesome-article. It's packed with useful info and insights.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Sed cursus ante dapibus diam. Sed nisi. Nulla quis sem at nibh elementum imperdiet. Duis sagittis ipsum. Praesent mauris. Fusce nec tellus sed augue semper porta. Mauris massa.

Nullam quis risus eget urna mollis ornare vel eu leo.
Integer posuere erat a ante venenatis dapibus posuere velit aliquet.
Special characters to consider: !@#$%^&*()_+-=[]{}|;':",.<>/?`~

Do you remember the last project's details? If not, here's a quick refresher: the budget was $500,000, and the timeline was 6 months. We faced several challenges, including supply chain issues and unexpected delays. Yet, we managed to deliver on time. Kudos to the team! 🎉

By the way, here’s a fun fact: the longest recorded flight of a chicken is 13 seconds! 🐔 How cool is that?

Our next team-building activity will be a hiking trip. Bring your gear and be ready for an adventure. 🌄

Hiking Essentials:
Sturdy boots 🥾
Water bottle 🚰
Snacks 🍫
First-aid kit 🩹
The goal is to reach the summit by noon. Let's make it happen, team! 💪
"""

In [86]:
preprocessed_text = preprocess_text(sample_text)
# preprocessed_text = preprocess_text(sample_text, stop_words=stopwords)
print(preprocessed_text)

Hey there! Have you checked out the new website? Visit us at or email us at for more details.

The meeting is scheduled for 12:30 PM on 15th August, 2023. Don't be late! We'll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%.

Here are the main points:

Increase marketing budget by 25%
Launch the new product line in Q4
Expand into the APAC region

To-do List:

 Finalize the budget proposal
 Review the Q3 report
 Prepare for the product launch event

Alice said, "We need to push our limits!" while Bob replied, "Let's ensure quality over quantity." This debate has been ongoing for weeks now. Some say, "Quality is key," but others argue, "We need quantity to drive growth."

Contact our support team at or call 1-800-555-0199. For international calls, dial +1-800-555-0199.

Here's a snippet of code we need to review:

def hello_world():
 print("Hello, World!")

Make sure to check it and ensure it's compliant with PEP 8 standards. Also, we have some le

In [87]:
nltk.sent_tokenize(preprocessed_text)

['Hey there!',
 'Have you checked out the new website?',
 'Visit us at or email us at for more details.',
 'The meeting is scheduled for 12:30 PM on 15th August, 2023.',
 "Don't be late!",
 "We'll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%.",
 'Here are the main points:\n\nIncrease marketing budget by 25%\nLaunch the new product line in Q4\nExpand into the APAC region\n\nTo-do List:\n\n Finalize the budget proposal\n Review the Q3 report\n Prepare for the product launch event\n\nAlice said, "We need to push our limits!"',
 'while Bob replied, "Let\'s ensure quality over quantity."',
 'This debate has been ongoing for weeks now.',
 'Some say, "Quality is key," but others argue, "We need quantity to drive growth."',
 'Contact our support team at or call 1-800-555-0199.',
 'For international calls, dial +1-800-555-0199.',
 'Here\'s a snippet of code we need to review:\n\ndef hello_world():\n print("Hello, World!")',
 "Make sure to check it and

In [90]:
parsed_text = stanza_tokenizer(preprocessed_text)

In [100]:
[sentence.text for sentence in parsed_text.sentences]

['Hey there!',
 'Have you checked out the new website?',
 'Visit us at or email us at for more details.',
 'The meeting is scheduled for 12:30 PM on 15th August, 2023.',
 "Don't be late!",
 "We'll discuss the Q3 financials, which include a profit margin of 12.5% and a YoY growth of 8%.",
 'Here are the main points:',
 'Increase marketing budget by 25%\nLaunch the new product line in Q4\nExpand into the APAC region',
 'To-do List:',
 'Finalize the budget proposal\n Review the Q3 report\n Prepare for the product launch event',
 'Alice said, "We need to push our limits!" while Bob replied, "Let\'s ensure quality over quantity."',
 'This debate has been ongoing for weeks now.',
 'Some say, "Quality is key," but others argue, "We need quantity to drive growth."',
 'Contact our support team at or call 1-800-555-0199.',
 'For international calls, dial +1-800-555-0199.',
 "Here's a snippet of code we need to review:",
 'def hello_world():\n print("Hello, World!")',
 "Make sure to check it and 

## gov-report

In [7]:
data_dir = "/Users/naman/Workspace/Data/UCCS-REU/GovReport"

crs_files = os.listdir(crs_dir := f"{data_dir}/crs")
gao_files = os.listdir(gao_dir := f"{data_dir}/gao")

print(f"crs files: {len(crs_files)}, gao files: {len(gao_files)}")

crs_out = f"{data_dir}/crs-processed"
gao_out = f"{data_dir}/gao-processed"

if not os.path.exists(crs_out):
	os.mkdir(crs_out)
if not os.path.exists(gao_out):
	os.mkdir(gao_out)

crs files: 7238, gao files: 12228


In [28]:
for file in crs_files:
	with open(f"{crs_dir}/{file}") as fp:
		data = json.load(fp)
	text = combine_subsections([data["reports"]])
	text = preprocess_text(text)
	summary = "\n".join(data["summary"])
	summary = preprocess_text(summary)
	with open(f"{crs_out}/{file}", "w") as fp:
		json.dump({
			"text": text,
			"summary": summary
		}, fp)

In [34]:
for file in gao_files:
	with open(f"{gao_dir}/{file}") as fp:
		data = json.load(fp)
	text = combine_subsections(data["report"])
	text = preprocess_text(text)
	print(data["highlight"])
	summary = "\n".join(data["highlight"])
	summary = preprocess_text(summary)
	with open(f"{gao_out}/{file}", "w") as fp:
		json.dump({
			"text": text,
			"summary": summary
		}, fp)

[]


---

In [4]:
save_dir = "/Users/naman/Workspace/Data/UCCS-REU/Models/BART"
checkpoint = "facebook/bart-large-cnn"

model = BartForCausalLM.from_pretrained(f"{save_dir}/model")
tokenizer = BartTokenizer.from_pretrained(f"{save_dir}/tokenizer")

In [60]:
print(inspect.signature(tokenizer))

(text: Union[str, List[str], List[List[str]]] = None, text_pair: Union[str, List[str], List[List[str]], NoneType] = None, text_target: Union[str, List[str], List[List[str]]] = None, text_pair_target: Union[str, List[str], List[List[str]], NoneType] = None, add_special_tokens: bool = True, padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False, truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = None, max_length: Optional[int] = None, stride: int = 0, is_split_into_words: bool = False, pad_to_multiple_of: Optional[int] = None, return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None, return_token_type_ids: Optional[bool] = None, return_attention_mask: Optional[bool] = None, return_overflowing_tokens: bool = False, return_special_tokens_mask: bool = False, return_offsets_mapping: bool = False, return_length: bool = False, verbose: bool = True, **kwargs) -> transformers.tokenization_utils_base.BatchEncodi

In [8]:
file = f"{crs_out}/{crs_files[0]}"

with open(file) as fp:
	data = json.load(fp)
data.keys()

dict_keys(['text', 'summary'])

In [119]:
inputs = tokenizer([data["text"]], return_tensors="pt")
inputs

{'input_ids': tensor([[    0, 43480, 24474,    35, 50118, 50118,   133, 11054,    22, 44189,
            12, 21426, 44195,   113,  3489, 12859,     7,     5, 10875,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [95]:
input_ids = torch.ones((1, 1024), dtype=int)
model(input_ids)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -5.6895,  -0.5144,   3.2866,  ...,  -1.2921,   0.0668,  -0.4549],
         [-36.3086,  -1.0934,   5.8454,  ...,   0.0640,   0.5099,   0.4333],
         [-32.5560,  -1.4362,   5.7458,  ...,  -0.3482,  -0.4919,  -1.1247],
         ...,
         [ -7.0467,  -1.8095,   5.4492,  ...,  -1.9251,  -3.2973,  -3.7877],
         [ -6.7949,  -1.8129,   5.4430,  ...,  -1.9251,  -3.2873,  -3.7860],
         [ -7.0110,  -1.8002,   5.5816,  ...,  -1.8608,  -3.2557,  -3.7590]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[ 6.2141e-01,  1.4915e+00,  3.5430e+00,  ...,  6.9152e-01,
            2.7157e+00, -1.6016e+00],
          [-1.0891e+00, -2.6220e+00, -1.6890e+00,  ...,  4.3597e+00,
            6.6120e-01,  1.1645e+00],
          [-3.8023e+00, -8.0654e-01, -2.1284e+00,  ..., -4.0281e+00,
           -5.6236e+00, -4.8699e+00],
          ...,
          [-1.3574e+00,  5.1857e+00,  8.9336e+00,  ..., -3.4450e-02,
        

In [62]:
text = data["text"]
sents = nltk.sent_tokenize(text)
len(sents)

287

In [57]:
context_size, _ = max_lengths(model)
context_size

1024

In [58]:
tokenized_sents = tokenizer(sents)
tokenized_sents

{'input_ids': [[0, 43480, 24474, 35, 50118, 50118, 133, 11054, 22, 44189, 12, 21426, 44195, 113, 3489, 12859, 7, 5, 10875, 9, 1861, 3454, 31, 915, 3454, 4, 2], [0, 1121, 42, 5377, 6, 1861, 3454, 12859, 7, 5, 1713, 4009, 11, 30, 8273, 39415, 3353, 6, 61, 42, 266, 67, 40, 9115, 7, 25, 22, 27045, 113, 50, 22, 33798, 1520, 72, 2], [0, 1121, 5709, 6, 915, 3454, 12859, 7, 1713, 4009, 11, 30, 5157, 8802, 8, 9196, 2566, 6, 61, 42, 266, 67, 12859, 7, 25, 22, 12406, 1757, 1520, 113, 50, 22, 8584, 31226, 2566, 72, 2], [0, 25997, 36689, 10, 10875, 9, 1861, 8, 915, 3454, 149, 237, 9042, 9, 5, 12539, 1783, 9, 26873, 104, 9041, 2485, 545, 6, 291, 6, 733, 6, 8, 2107, 6025, 58, 1887, 22, 560, 2097, 5, 22315, 24112, 9, 1188, 88, 21779, 1414, 17220, 113, 1216, 237, 17947, 7668, 32, 10266, 4997, 7, 25, 5, 10352, 12, 21426, 44195, 1783, 4, 2], [0, 45875, 6, 7162, 733, 9986, 786, 27045, 31, 8394, 8874, 6, 150, 37028, 545, 6, 291, 6, 8, 2107, 9986, 8273, 39415, 3353, 31, 36758, 1295, 19, 5157, 2566, 8, 31, 7

In [93]:
mean_length = np.mean([
	len(sent) for sent in tokenized_sents["input_ids"]
	])
mean_length

38.930313588850176

In [94]:
num_sents = int(context_size / mean_length)
num_sents

26

In [96]:
inputs = pick_sents(tokenized_sents["input_ids"], num_sents)
inputs.shape

torch.Size([1022])

In [112]:
pick_sents(data["text"], nltk.sent_tokenize, tokenizer, context_size)

tensor([    0, 10462,     6,   613, 16645,     8,  3454,  5730,  2857,  2322,
          484,   796,     8,  3947,   210,  6795,    19,   182,   430,  4099,
            8,  4107,  8369,   148,     5,  4525,    29,     4,     2,     0,
        10462,     6,    11,  3478, 14255,    11,  8008,     6,     5,  2337,
         1130,     5,  4200,   903, 11543,  3000,     7,   564,  2153,     2,
            0,  1121,  1285,     6,     5, 18753,    12, 16025,  1783, 29820,
         5501,   223, 13293,     6, 26225,  8111,     6,     8, 15636,   710,
          405,  1938,  2820,    13,   258,  8273, 39415,     8, 29470,  2462,
        39415,  3353,    11,    41,  2120,     7,  1100,     5,  5501,    12,
         3368,   743,    14,  2551,     7,   310,   215,    10,  1233,   774,
           11,     5,   485,   613,  1486,     4,     2,     0,  3972,     5,
         5239,    14,  1520,   946,  1233,  5353,     9,  4732,  5157,    50,
           97, 15325,  1781,     6,    10,  6143, 22380,  9762, 